### 1. testing how python supports competitors
some functions like `__eq__` has opposite operator, and some other operations are not
handleable by left side (default) so like for `a + b` (`a.__add__(b)`) that if a doesn't
have `__add__(...)`, python tries `b.__radd__(a)`. here testing those properties

the way to figure it out is simple actually, we let it run repeatedly and simply delete
the last used method so it have to try a new way until it can't no more.

In [49]:
from typing import Any


# creates a dummy method that instead of acting as a comparison operator
# is a logger that after the first call deletes itself from the class
def one_time_callable(name):
    def wrapped(self, *args, **kwargs) -> Any:
        str_args = tuple(map(str, args))
        str_kwargs = tuple(map('{}={}'.format, kwargs.items()))
        call_props = '(%s)' % ', '.join(str_args + str_kwargs)
        print(self.name, name, sep='.', end=call_props + '\n')

        delattr(test, name)

    return wrapped


# a simple class that will be used to test the comparison operators
# which certainly contains any needed methods to be compared
class test:
    def __init__(self, name):
        self.name = name

    def __str__(self) -> str:
        return self.name

    __lt__ = one_time_callable("__lt__")
    __le__ = one_time_callable("__le__")
    __eq__ = one_time_callable("__eq__")
    __ne__ = one_time_callable("__ne__")
    __gt__ = one_time_callable("__gt__")
    __ge__ = one_time_callable("__ge__")


i = 1
while True:
    print(f'attempt:', i)
    try:
        test('a') > test('b')  # type: ignore
    except TypeError:
        print('failed')
        break
    print()
    i += 1

attempt: 1
a.__gt__(b)

attempt: 2
b.__lt__(a)

attempt: 3
failed
